<a href="https://colab.research.google.com/github/tiagoandremv/MachineLearning/blob/main/FaceUNMASKER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importação de bibliotecas

In [1]:
# IMPORTAÇÃO DE BIBLIOTECAS NECESSÁRIAS
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, mean_squared_error
from sklearn.model_selection import train_test_split
from PIL import Image

# usando o cv2, deixa de ser necesasrio fazer o import do google drive
#from google.colab import drive
#drive.mount('/gdrive')

#Definição de funções

In [2]:
def __init__(self):
  path = '/content/drive/My Drive/Colab Notebooks/tdx_project/dataset-proj/'
  self.train_path = path + 'train/'  
  self.test_path = path + 'validation/' 

Função para visualizar progressão da imagem de treino:

In [3]:
def plot_all_images():
  for i in range(len(images)):
    plt.subplot(1,len(images),i+1) 
    plt.imshow(images[i])
    plt.title(titles[i])
    
    plt.xticks([])
    plt.yticks([])
  plt.show()

def plot_image(img):
  plot = plt.figure()
#  plot = plot.add_subplot(4, 5, num + 1)
  plot.imshow(img)
  plot.show()



Função para realizar o rescale da imagem:

In [4]:
def scale_image(img, scale_percent = 50):
  width = int(img.shape[1] * scale_percent / 100)
  height = int(img.shape[0] * scale_percent / 100)
  scale = (width, height)
  img = cv2.resize(img, scale)
  return img

In [5]:
def label_img(path):
    word_label = path.split('/')[-2]
    # DIY One hot encoder
    if word_label == 'fake':
        return [1]
    elif word_label == 'real':
        return [0]

In [6]:
label_img('/content/drive/My Drive/Colab Notebooks/tdx_project/dataset/train/fake/159.jpg')

[1]

#Carregamento de Imagens

In [7]:
from tqdm import tqdm # permite verificar o progresso do carregamento
from random import shuffle

def load_prepare_data (_path):
  
  dataset = []
  
  for folder in os.listdir(_path): #['fake', 'real']
    
    path_folder = os.path.join(_path, folder)

    for filename in tqdm(os.listdir(path_folder)):

      
      #print(path_folder)
      path = os.path.join(path_folder, filename)
      #print(path)
      label = label_img(path) # [0,1] para real , [1,0] para fake
      #print(label)

      img = cv2.imread(path)
      
      img = scale_image(img)
      img = scale_image(img)

      #img = img.astype("float64") / 255.0
      #original_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
      
      dataset.append([np.array(img), np.array(label)])
      #break
  
  shuffle(dataset)
  return dataset




In [8]:
train_data = load_prepare_data('/content/drive/My Drive/Colab Notebooks/tdx_project/dataset-proj/train/')

100%|██████████| 700/700 [00:54<00:00, 12.76it/s]


In [9]:
print(np.asarray(train_data).shape)

(1400, 2)


In [10]:
num, data = train_data[0]

inputs_shape = num.shape
print("Inputs Shape: ", inputs_shape)

Inputs Shape:  (256, 256, 3)


# Definição do MODEL

In [28]:
from tensorflow.keras import layers, models

model = models.Sequential()

### YOUR CODE HERE: ~9 lines of code expected, one per layer ### 
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=inputs_shape))
model.add(layers.MaxPooling2D((2, 2)))
#model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
#model.add(layers.Dropout(0.2))

model.add(layers.Flatten())

model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(10))

In [31]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy, binary_crossentropy

### YOUR CODE HERE ###
model.compile(optimizer='adam', 
              loss=SparseCategoricalCrossentropy(True), 
              metrics=['accuracy'])

Definição de features

In [22]:
# X-Features & Y-Labels
train_images = np.array([i[0] for i in train_data])
train_labels = np.array([i[1] for i in train_data]).astype('uint8')

#train_images[1][1].reshape(-1, 512, 512, 1)

print(np.asarray(train_data).shape)
print(train_images.shape, train_images.dtype)
print(train_labels.shape, train_labels.dtype)

(1400, 2)
(1400, 256, 256, 3) uint8
(1400, 1) uint8


In [23]:
train_labels

array([[0],
       [1],
       [0],
       ...,
       [0],
       [0],
       [0]], dtype=uint8)

In [33]:
history = model.fit(train_images, train_labels, epochs = 50, validation_split=0.2)

Epoch 1/50
35/35 [==============================] - 2s 63ms/step - loss: 0.1295 - accuracy: 0.9348 - val_loss: 1.9353 - val_accuracy: 0.5357
Epoch 2/50
35/35 [==============================] - 2s 60ms/step - loss: 0.1206 - accuracy: 0.9509 - val_loss: 2.0418 - val_accuracy: 0.5679
Epoch 3/50
35/35 [==============================] - 2s 59ms/step - loss: 0.1031 - accuracy: 0.9580 - val_loss: 2.1161 - val_accuracy: 0.5571
Epoch 4/50
35/35 [==============================] - 2s 60ms/step - loss: 0.1254 - accuracy: 0.9554 - val_loss: 2.3252 - val_accuracy: 0.5500
Epoch 5/50
35/35 [==============================] - 2s 60ms/step - loss: 0.0887 - accuracy: 0.9661 - val_loss: 2.0209 - val_accuracy: 0.5393
Epoch 6/50
35/35 [==============================] - 2s 60ms/step - loss: 0.0619 - accuracy: 0.9759 - val_loss: 2.8025 - val_accuracy: 0.5714
Epoch 7/50
35/35 [==============================] - 2s 60ms/step - loss: 0.0465 - accuracy: 0.9830 - val_loss: 3.8431 - val_accuracy: 0.5643
Epoch 8/50
35

In [ ]:
train_labels

In [ ]:
# APRESENTAÇÃO DE RESULTADOS

#print("Confusion Matrix\n", confusion_matrix(y_train, train_predictions))
#print("Accuracy: %.2f %%" % (accuracy_score(y_train, train_predictions) * 100))